In this simple example we configure our `thermocalc` 
model entirely from the script without utilising 
configuration files. 

We walk through various features of the `tawnycalc` 
configuration, execution, and data extraction. 

In [1]:
import os
os.environ["THERMOCALC_EXECUTABLE"] = "/home/jmansour/code/thermocalc/thermo"

In [2]:
import tawnycalc as tc

In [3]:
# To start with a clean context, set `scripts_dir=None`
context = tc.Context(scripts_dir=None)
context.prefs

calcmode   : 1
scriptfile : warfyh
dataset    : None

In [4]:
# Set the required dataset on the `prefs` dictionary.
context.prefs["dataset"] = 62

In [5]:
# Configure some parameters directly on the `script` dictionary.
help(tc.Context.script)

Help on property:

    The `script` dictionary contains your `thermocalc` model configuration. 
    This dictionary will be populated with the data from your `thermocalc` 
    scripts where the `scripts_dir` parameter is passed when constructing
    your `Context`.  Otherwise it will be empty and you will be required 
    to sufficiently populate it.  In either case, you may add, modify and
    delete values as necessary before calling `execute`. 
    
    Please see `help(tawnycalc.data_objects.thermocalc_script)` for further
    info on script object usage.



In [6]:
help(tc.data_objects.thermocalc_script)

Help on class thermocalc_script in module tawnycalc.data_objects:

class thermocalc_script(collections.OrderedDict)
 |  This dictionary class generates objects which represent `thermocalc` scripts 
 |  and provides associated methods to handle the script lifecycle and user 
 |  interaction.
 |  
 |  Key/Value pairs will be directly printed to form standard `thermocalc`
 |  input.  For example 
 |  
 |  >>> context.script[       "axfile"] = "mb50NCKFMASHTO"
 |  >>> context.script[        "which"] = "chl bi pa ep ru chl g ilm sph"
 |  >>> context.script[     "inexcess"] = "mu q H2O"
 |  >>> context.script[       "dogmin"] = "yes 0"
 |  
 |  is equivalent to
 |  
 |      axfile   mb50NCKFMASHTO
 |      which    chl bi pa ep ru chl g ilm sph
 |      inexcess mu q H2O
 |      dogmin   yes 0
 |  
 |  To form repeat keyword entries, set the required corresponding values 
 |  in a Python list.  For example:
 |  
 |  >>> context.script[   "samecoding"] = ["mu pa", "sp mt"]
 |  
 |  is equivalen

In [7]:
context.script[       "axfile"] = "mb50NCKFMASHTO"
context.script[         "with"] = "chl bi pa ru g ilmm sph"
context.script[     "inexcess"] = "mu q H2O"
context.script[       "dogmin"] = "1"                 # replaces 'dogmin yes'
context.script[       "maxvar"] = "6"                 # replaces 'setmaxvar'
context.script["pseudosection"] = ""
context.script[   "samecoding"] = ["mu pa", "sp mt"]
context.script[    "diagramPT"] = "2 20 400 1100"     # new script
context.script[        "calcP"] = 11.0
context.script[        "calcT"] = 600.0

In [8]:
# Add 'rbi' section.
# We have specific `rbi` object that we use to achieve this. 
# Let's create rbi object, noting how we provide the oxide list at construction time:
rbi_sec = tc.rbi(oxides="H2O SiO2 Al2O3 CaO MgO FeO K2O Na2O TiO2 O")

In [9]:
# Now we need to add the individual phases.
# We will use the `add_phase` method to achieve this. 
# We supply the `phase` name, `mode` data, and finally
# `oxides` as a string or list
help(rbi_sec.add_phase)

Help on method add_phase in module tawnycalc.data_objects:

add_phase(phase, mode, oxides) method of tawnycalc.data_objects.rbi instance
    Add a phase to the rbi table. 
    
    Params
    ------
    phase:  str
        Name of phase to add. 
    mode: str, float
        Mode/proportion of phase.
    oxides: str,list
        Proportion of each oxide for phase.



In [10]:
rbi_sec.add_phase(  "g",  "0.0"     , "       0    3.000000    0.984049    0.230147    0.395429    2.406326           0           0           0    0.015951")
rbi_sec.add_phase( "mu",  "0.159799", "1.000000    3.102219    1.393080    0.004483    0.058473    0.053149    0.386538    0.111220           0    0.002460")
rbi_sec.add_phase( "pa",  "0.006915", "       1    2.984010    1.505704    0.019628    0.001993    0.002589    0.033770    0.456416           0    0.000473")
rbi_sec.add_phase( "bi",  "0.090237", "0.932353    2.842048    0.636750           0    1.244448    1.572357    0.500000           0    0.067647    0.021202")
rbi_sec.add_phase("ilm",  "0.004242", "       0           0           0           0    0.011126    1.032649           0           0    0.956225    0.043775")
rbi_sec.add_phase(  "q",  "0.240992", "       0           1           0           0           0           0           0           0           0           0")
rbi_sec.add_phase("H2O",  "0.458826", "       1           0           0           0           0           0           0           0           0           0")

In [11]:
# Let's have a look at the object
rbi_sec

               H2O       SiO2      Al2O3     CaO       MgO       FeO       K2O       Na2O      TiO2      O
g    0.0       0.0       3.0       0.984049  0.230147  0.395429  2.406326  0.0       0.0       0.0       0.015951
mu   0.159799  1.0       3.102219  1.39308   0.004483  0.058473  0.053149  0.386538  0.11122   0.0       0.00246
pa   0.006915  1.0       2.98401   1.505704  0.019628  0.001993  0.002589  0.03377   0.456416  0.0       0.000473
bi   0.090237  0.932353  2.842048  0.63675   0.0       1.244448  1.572357  0.5       0.0       0.067647  0.021202
ilm  0.004242  0.0       0.0       0.0       0.0       0.011126  1.032649  0.0       0.0       0.956225  0.043775
q    0.240992  0.0       1.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0
H2O  0.458826  1.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0

In [12]:
# You can also access individual values directly
rbi_sec["g"]["Al2O3"]

0.984049

In [13]:
# Values can also be modified
rbi_sec["g"]["Al2O3"] = 0.90
# Print again to confirm:
rbi_sec["g"]["Al2O3"]

0.9

In [14]:
# A point of subtlety... you may be noticed 
# above in the first instance we were returned
# '0.984049' which is actually a string, and 
# in the second instance we were returned 0.9,
# which is a float. Generally, you can use strings
# and their equivalent float interchangeably.

In [15]:
# Let's set that value back now
rbi_sec["g"]["Al2O3"] = 0.984049

In [16]:
# Ok, we have successfully created the `rbi` object, 
# but don't forget that it must be set on the `script` for
# it to take effect:
context.script["rbi"] = rbi_sec

In [17]:
# Now create xyzguess section.  
# We use the `xyz` object this time. 
# It is really just a Python `OrderedDict`.
# First create a dictionary to record values
xyzguess = tc.xyz()
# Now add values
xyzguess[    "x(g)"] = "0.885508"
xyzguess[    "z(g)"] = "0.211255"
xyzguess[    "f(g)"] = "0.070810"
xyzguess[   "x(bi)"] = "0.579607"
xyzguess[   "y(bi)"] = "0.140313"
xyzguess[   "f(bi)"] = "0.126258"
xyzguess[   "t(bi)"] = "0.072042"
xyzguess[   "Q(bi)"] = "0.121282"
xyzguess[   "x(mu)"] = "0.461998"
xyzguess[   "y(mu)"] = "0.944788"
xyzguess[   "f(mu)"] = "0.009495"
xyzguess[   "n(mu)"] = "0.253509"
xyzguess[   "c(mu)"] = "0.016160"
xyzguess[   "x(pa)"] = "0.461998"
xyzguess[   "y(pa)"] = "0.996868"
xyzguess[   "f(pa)"] = "0.002376"
xyzguess[   "n(pa)"] = "0.955475"
xyzguess[   "c(pa)"] = "0.038175"
xyzguess[   "x(ma)"] = "0.461998"
xyzguess[   "y(ma)"] = "0.964236"
xyzguess[   "f(ma)"] = "0.009701"
xyzguess[   "n(ma)"] = "0.054672"
xyzguess[   "c(ma)"] = "0.944410"
xyzguess[  "x(chl)"] = "0.473331"
xyzguess[  "y(chl)"] = "0.520964"
xyzguess[  "f(chl)"] = "0.248348"
xyzguess["QAl(chl)"] = "0.230540  range -1.000 1.000"
xyzguess[ "Q1(chl)"] = "0.077845  range -1.000 1.000"
xyzguess[ "Q4(chl)"] = "0.092301  range -1.000 1.000"
xyzguess[   "f(ep)"] = "0.264635"
xyzguess[   "Q(ep)"] = "0.250072  range  0.000 0.500"
xyzguess[  "i(ilm)"] = "0.829808"
xyzguess[  "g(ilm)"] = "0.017481"
xyzguess[  "Q(ilm)"] = "0.760053  range -0.990 0.990"
xyzguess[  "x(mt1)"] = "0.762081"
xyzguess[  "Q(mt1)"] = "0.808179"

In [18]:
# Again, don't forget to set on the script:
context.script["xyzguess"] = xyzguess

In [19]:
# Now we have a complete configuration, let's execute. 
results = context.execute()

In [20]:
context.print_output()

THERMOCALC 3.50 (Free Pascal version)

summary output in the file, "tc-warfyh-o.txt"
other (eg drawpd) output in the file, "tc-warfyh-dr.txt"
details of calc results in the file, "tc-warfyh-ic.txt"
initial tables in the file, "tc-warfyh-it.txt"
csv format in the file, "tc-warfyh.csv"
more csv format in the file, "tc-warfyh2.csv"
(these files may not all be populated yet, depending on the calcs;
 thermocalc should delete empty files at the end of each run)
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

THERMOCALC 3.50 running at 14.30 on Wed 16 Dec,2020
using tc-ds62.txt produced at 20.08 on Mon 6 Feb,2012
with axfile tc-mb50NCKFMASHTO.txt and scriptfile tc-warfyh.txt

reading ax: L hb aug dio opx g ol ksp plc pli sp ilm ilmm ep bi mu chl ru sph q 
            an ab H2O 

with: g ilmm bi mu pa chl ru sph q H2O  (from script)
fluid is just H2O
in excess: mu q H2O

specification of xyz starting guesses of phases
in the scriptfile: g bi mu pa chl; not in the scriptfile: ilmm

In [21]:
# Ok, we have a successful execution. 
# The resulting data is recorded in the returned
# dictionary object.  Execute the `print_keys()` method
# to see what is available. Note that keys prepended 
# with `output_` provide the raw output which may be useful 
# at times.
results.print_keys()

P
T
bulk_composition
modes
output_tc_ic
phases
rbi
site_fractions
thermodynamic_properties
xyz


In [22]:
# Let's print the data. Note that most of the returned
# data objects are based in Python dictionaries. 
# You can therefore (where appropriate) also access 
# values directly.

In [23]:
results.P

11.0

In [24]:
results.T

600.0

In [25]:
results.bulk_composition

H2O    52.1705
SiO2   36.9434
Al2O3   4.3466
CaO     0.0129
MgO     1.7707
FeO     2.461
K2O     1.5886
Na2O    0.3156
TiO2    0.3454
O       0.0455

In [26]:
results.bulk_composition["H2O"]

52.1705

In [27]:
results.modes

g     0.0002
ilmm  0.0037
bi    0.096
mu    0.1645
pa    0.0066
ru    0.0001
q     0.251
H2O   0.4778

In [28]:
results.modes["g"]

0.0002

In [29]:
results.phases

'g ilmm bi pa ru (mu, q, fluid) or [chl,sph] #71'

In [30]:
results.rbi

                H2O       SiO2      Al2O3     CaO       MgO       FeO       K2O       Na2O      TiO2      O
g     0.00023   0.0       3.0       0.986249  0.152574  0.413022  2.461905  0.0       0.0       0.0       0.013751
ilmm  0.003702  0.0       0.0       0.0       0.0       0.025779  1.019766  0.0       0.0       0.954455  0.045545
bi    0.096041  0.908144  2.83887   0.639339  0.0       1.220537  1.570058  0.5       0.0       0.091856  0.021791
mu    0.164535  1.0       3.094055  1.401276  0.004393  0.053634  0.049759  0.383851  0.113953  0.0       0.002473
pa    0.006642  1.0       2.984245  1.505707  0.019132  0.00184   0.002501  0.034413  0.456021  0.0       0.000482
ru    7.1e-05   0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       1.0       0.0
q     0.250991  0.0       1.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0
H2O   0.477788  1.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0

In [31]:
results.rbi["mu"]["CaO"]

0.004393

In [32]:
results.site_fractions

g     xMgX     xFeX     xCaX     xAlY     xFe3Y
      0.13767  0.81147  0.05086  0.98625  0.01375
ilmm  xFeA     xTiA     xMgA     xFe3A    xFeB     xTiB     xFe3B
      0.90969  0.01898  0.02578  0.04554  0.01898  0.93547  0.04554
bi    xMgM3    xFeM3    xFe3M3   xTiM3    xAlM3    xMgM12   xFeM12   xSiT     xAlT     xOHV     xOV
      0.24316  0.50385  0.04358  0.09186  0.11755  0.48869  0.51131  0.41944  0.58056  0.90814  0.09186
mu    xKA      xNaA     xCaA     xMgM2A   xFeM2A   xAlM2A   xAlM2B   xFe3M2B  xSiT1    xAlT1
      0.76770  0.22791  0.00439  0.05363  0.04481  0.90155  0.99505  0.00495  0.54703  0.45297
pa    xKA      xNaA     xCaA     xMgM2A   xFeM2A   xAlM2A   xAlM2B   xFe3M2B  xSiT1    xAlT1
      0.06883  0.91204  0.01913  0.00184  0.00154  0.99662  0.99904  0.00096  0.49212  0.50788

In [33]:
results.site_fractions["bi"]["xAlM3"]

'0.11755'